In [ ]:
############improved version always second
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

path = "/content/drive/MyDrive/dataset/raw data.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['CR (%)']) #output feature
X = data.drop(['CR (%)', 'ECR PY (%)', 'PDOs PY (%)', 'POHs PY (%)', 'V'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

model = DecisionTreeRegressor(criterion="squared_error", max_depth=None, max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2)
model.fit(imp_train_X, train_y)
preds_val = model.predict(imp_test_X)

score = model.score(imp_train_X,train_y)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))



Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Training score:  0.9999978961344894
Mean Absolute Error: 0.16013297575966134
MSE: 0.09128333743769917
RMSE: 0.3021313248203489
R2 score: 0.9128263327112014


In [ ]:
#gridsearch next!

import pandas as pd
import numpy as np
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

path = "/content/drive/MyDrive/dataset/raw data.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['PDOs PY (%)']) #output feature
X = data.drop(['CR (%)', 'PDOs PY (%)', 'I (A)'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

#hyperparamter optimization using gridsearch
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
random_cv = GridSearchCV(
            estimator=DecisionTreeRegressor(),
            param_grid={'max_depth': [None, 5, 10],
                        'min_samples_split': [2, 5, 10],
                        'min_samples_leaf': [1, 5, 10],
                        'max_leaf_nodes': [None, 20, 50],
                        'criterion': ['friedman_mse', 'absolute_error', 'squared_error']}, #, 'poisson'
            cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
#This includes 3-fold validation (cv=3) and uses mean scare error as the metric for evaluation

random_cv.fit(imp_train_X, train_y)

# use the best parameters
best_params = random_cv.best_params_
best_score = random_cv.best_score_
print('Best hyperparameters:', best_params)

# using the best parameters run XGBoost
model = DecisionTreeRegressor(**best_params)
model.fit(imp_train_X, train_y)

score = model.score(imp_train_X,train_y)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))

Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Best hyperparameters: {'criterion': 'absolute_error', 'max_depth': 10, 'max_leaf_nodes': 50, 'min_samples_leaf': 1, 'min_samples_split': 2}
Training score:  0.680243490118653
Mean Absolute Error: 0.2529876359135884
MSE: 0.4449198916432459
RMSE: 0.6670231567518822
R2 score: 0.6049778550726286


In [ ]:
############improved version always second (PDOs PY)

import pandas as pd
import numpy as np
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

path = "/content/drive/MyDrive/dataset/raw data.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['ECR PY (%)']) #output feature
X = data.drop(['CR (%)', 'ECR PY (%)', 'PDOs PY (%)', 'POHs PY (%)', 'V'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

model = DecisionTreeRegressor(criterion="absolute_error", max_depth=10, max_leaf_nodes=50, min_samples_leaf=1, min_samples_split=2)
model.fit(imp_train_X, train_y)
preds_val = model.predict(imp_test_X)

score = model.score(imp_train_X,train_y)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))


Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Training score:  0.6806977834260924
Mean Absolute Error: 0.24625488380417768
MSE: 0.44192680904077103
RMSE: 0.6647757584635371
R2 score: 0.6077365280751355
